In [1]:
import torchvision
import copy

from lightly.models.modules import DINOProjectionHead
from lightly.data import LightlyDataset
from lightly.transforms.dino_transform import DINOTransform
from lightly.models.utils import deactivate_requires_grad, update_momentum

import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import torchmetrics

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
batch_size = 24
num_frames = 22

device_ids = [0,1,2,3]

In [4]:
import os
import numpy as np
import glob

# list all directories in the current directory
directories = glob.glob("/scratch/pt2295/Dataset_Student/train/video_*")#[d for d in os.listdir('/scratch/pt2295/Dataset_Student/train/') if os.path.isdir(d)]

# create an empty list to store numpy arrays
arrays = {}
# print(directories)

# loop through each directory
ctr=0
for d in directories:
    # construct the path to the mean.npy file
    path_to_mean = os.path.join(d, 'mask.npy')
    # read the mean.npy file as a numpy array
    mean_array = np.load(path_to_mean)
    print(mean_array.shape)
    # append the mean array to the list of arrays
    arrays[ctr]=mean_array
    ctr+=1
    break




In [5]:
len(directories)

0

### Dataloader

In [6]:
def one_hot_encoder(mask, num_classes=49):
  # mask: a numpy array of shape (batch, height, width) with integer class labels
  # num_classes: an integer representing the number of classes
  # returns: a numpy array of shape (batch, height, width, num_classes) with one-hot encoded labels
    one_hot = np.eye(num_classes)[mask] # select rows from identity matrix
    return one_hot

In [7]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset

class MeanNpyDataset(Dataset):
    def __init__(self, directory_path):
        self.directory_path = directory_path
        self.folder_names = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

    def __len__(self):
        return len(self.folder_names)

    def __getitem__(self, idx):
        folder_name = self.folder_names[idx]
       
        path_to_mean = os.path.join(self.directory_path, folder_name, 'mask.npy')

        mean_array = np.load(path_to_mean)
       
        mean_array=one_hot_encoder(mean_array)
        

        return torch.from_numpy(mean_array), folder_name


In [8]:
dataset = MeanNpyDataset('/scratch/jz3395/train')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)



In [9]:
for batch_idx, (mean_tensor, folder_name) in enumerate(dataloader):
    # do something with the batch of mean tensors and corresponding folder names
#     print(f"Batch {batch_idx}, folder names: {folder_name}")
    print(mean_tensor.shape)
    break

torch.Size([24, 22, 160, 240, 49])


In [10]:
import torch
import torch.nn as nn

class ConvLSTMCell(nn.Module):

    def __init__(self, in_channels, out_channels, 
    kernel_size, padding, activation, frame_size):

        super(ConvLSTMCell, self).__init__()  

        if activation == "tanh":
            self.activation = torch.tanh 
        elif activation == "relu":
            self.activation = torch.relu
        
        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        self.conv = nn.Conv2d(
            in_channels=in_channels + out_channels, 
            out_channels=4 * out_channels, 
            kernel_size=kernel_size, 
            padding=padding)           

        # Initialize weights for Hadamard Products
        self.W_ci = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_co = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_cf = nn.Parameter(torch.Tensor(out_channels, *frame_size))

    def forward(self, X, H_prev, C_prev):

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        conv_output = self.conv(torch.cat([X, H_prev], dim=1))

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        i_conv, f_conv, C_conv, o_conv = torch.chunk(conv_output, chunks=4, dim=1)

        input_gate = torch.sigmoid(i_conv + self.W_ci * C_prev )
        forget_gate = torch.sigmoid(f_conv + self.W_cf * C_prev )

        # Current Cell output
        C = forget_gate*C_prev + input_gate * self.activation(C_conv)

        output_gate = torch.sigmoid(o_conv + self.W_co * C )

        # Current Hidden State
        H = output_gate * self.activation(C)

        return H, C

class ConvLSTM(nn.Module):

    def __init__(self, in_channels, out_channels, 
    kernel_size, padding, activation, frame_size):

        super(ConvLSTM, self).__init__()

        self.out_channels = out_channels

        # We will unroll this over time steps
        self.convLSTMcell = ConvLSTMCell(in_channels, out_channels, 
        kernel_size, padding, activation, frame_size)

    def forward(self, X):

        # X is a frame sequence (batch_size, num_channels, seq_len, height, width)

        # Get the dimensions
        batch_size, _, seq_len, height, width = X.size()

        # Initialize output
        output = torch.zeros(batch_size, self.out_channels, seq_len, 
        height, width, device=device)
        
        # Initialize Hidden State
        H = torch.zeros(batch_size, self.out_channels, 
        height, width, device=device)

        # Initialize Cell Input
        C = torch.zeros(batch_size,self.out_channels, 
        height, width, device=device)

        # Unroll over time steps
        for time_step in range(seq_len):

            H, C = self.convLSTMcell(X[:,:,time_step], H, C)

            output[:,:,time_step] = H

        return output
    
class Seq2Seq(nn.Module):

    def __init__(self, num_channels, num_kernels, kernel_size, padding, 
    activation, frame_size, num_layers):

        super(Seq2Seq, self).__init__()

        self.sequential = nn.Sequential()

        # Add First layer (Different in_channels than the rest)
        self.sequential.add_module(
            "convlstm1", ConvLSTM(
                in_channels=num_channels, out_channels=num_kernels,
                kernel_size=kernel_size, padding=padding, 
                activation=activation, frame_size=frame_size)
        )

        self.sequential.add_module(
            "batchnorm1", nn.BatchNorm3d(num_features=num_kernels)
        ) 

        # Add rest of the layers
        for l in range(2, num_layers+1):

            self.sequential.add_module(
                f"convlstm{l}", ConvLSTM(
                    in_channels=num_kernels, out_channels=num_kernels,
                    kernel_size=kernel_size, padding=padding, 
                    activation=activation, frame_size=frame_size)
                )
                
            self.sequential.add_module(
                f"batchnorm{l}", nn.BatchNorm3d(num_features=num_kernels)
                ) 

        # Add Convolutional Layer to predict output frame
        self.conv = nn.Conv2d(
            in_channels=num_kernels, out_channels=num_channels,
            kernel_size=kernel_size, padding=padding)

    def forward(self, X):

        # Forward propagation through all the layers
        output = self.sequential(X)

        # Return only the last output frame
        output = self.conv(output[:,:,-1])
        
        return nn.Softmax()(output)
    

In [11]:
class_weight=torch.eye(1,1)
print(class_weight.size())
full=torch.full((1,48),49)
print(full.size())
class_weight=torch.cat((class_weight,full),dim=1)
print(class_weight)

torch.Size([1, 1])
torch.Size([1, 48])
tensor([[ 1., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90.,
         90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90.,
         90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90., 90.,
         90., 90., 90., 90., 90., 90., 90.]])


In [25]:
# ckpt = torch.load('convlstm_predictor.pth')
# predictor.load_state_dict(ckpt)

predictor = Seq2Seq(num_channels=49, num_kernels=64, kernel_size=(3, 3), padding=(1, 1), activation="relu", 
                frame_size=(160, 240), num_layers=3)

predictor = nn.DataParallel(predictor.cuda(), device_ids=device_ids).cuda()

optimizer = torch.optim.Adam(predictor.parameters(), lr=1e-2)

criterion = nn.CrossEntropyLoss(weight=class_weight.to(device),reduction='mean')
starting_epoch=89
checkpoint = torch.load('/scratch/jz3395/p_convlstm_predictor'+str(starting_epoch)+'.pth')
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=5)
predictor.load_state_dict(checkpoint)
jaccard= torchmetrics.JaccardIndex(task="multiclass", num_classes=49).to(device)

In [26]:
len(dataloader)

42

In [27]:
def train_for_11_frames_input():
    num_epochs = 60
    t1 = datetime.now()
    for e in range(num_epochs):
        total_loss = 0
        correct=0
        total=0
        correct_last_frame_prediction=0
        total_last_frame_prediction=0
        jaccard_index=0
        for i, (true_masks, folder_name) in enumerate(dataloader):
            print(i,end=' ')
            #print("avg IoU last image:",jaccard_index)
            optimizer.zero_grad()
            true_masks=true_masks.to(torch.float32).permute(0,4,1,2,3).to(device)
            #true_masks is of dimension here:([16, 49, 22, 160, 240])
            first_11_true_frames = true_masks[:,:,:11,:,:]
            last_11_true_frames=true_masks[:,:,11:,:,:]
            pred_mask=predictor(first_11_true_frames)
            
            for i in range(0,11):
                #pred mask is of dimension torch.Size([16, 49, 160, 240])
                frame_of_true_mask=true_masks.argmax(dim=1).squeeze()[:, 11+i, :,:]
                loss = criterion(pred_mask,frame_of_true_mask)
                new_correct=torch.sum(pred_mask.argmax(dim=1).squeeze()==frame_of_true_mask)
                new_total=batch_size*160*240
                correct+=new_correct
                total+=new_total
                if i==10:
                    total_last_frame_prediction+=new_total
                    correct_last_frame_prediction+=new_correct
                    #jaccard_index=jaccard(pred_mask.argmax(dim=1).squeeze(),frame_of_true_mask)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                if i!=10:
                    next_mask = last_11_true_frames[:, :, i, :, :]
                    next_mask=next_mask.unsqueeze(2)
                    pred_mask= predictor(next_mask)
            
        total_epoch=e+starting_epoch
        torch.save(predictor.state_dict(), '/scratch/jz3395/p_convlstm_predictor'+str(total_epoch+1)+'.pth')
        
        print(f"\n\nEpoch: {total_epoch}, Loss: {total_loss}\n, overall prediction accuracy: {correct/total}")
        print(f"\n\nEpoch: {total_epoch}, Prediction accuracy of last frame: {correct_last_frame_prediction/total_last_frame_prediction}")
    print(f"Training took time: {datetime.now() - t1}")
                
            



In [28]:
!jupyter trust convlstm.ipynb

Notebook already signed: convlstm.ipynb


In [ ]:
train_for_11_frames_input()

0 

/state/partition1/job-32887502/ipykernel_2858881/530924047.py:138: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.Softmax()(output)


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 

Epoch: 89, Loss: 1515.6507575511932
, overall prediction accuracy: 0.934627115726471


Epoch: 89, Prediction accuracy of last frame: 0.9425966739654541
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 

Epoch: 90, Loss: 1556.3480298519135
, overall prediction accuracy: 0.9359146952629089


Epoch: 90, Prediction accuracy of last frame: 0.9451081156730652
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 

Epoch: 91, Loss: 1537.0702075958252
, overall prediction accuracy: 0.9387143850326538


Epoch: 91, Prediction accuracy of last frame: 0.9446868300437927
0 1 2 3 4 5 6 7 8 9 10 11 

In [16]:
dataset = MeanNpyDataset('/scratch/jz3395/train')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
def validate_for_11_frames_input():
    num_epochs = 50
    t1 = datetime.now()
    for e in range(1):
        total_loss = 0
        correct=0
        total=0
        correct_last_frame_prediction=0
        total_last_frame_prediction=0
        jaccard_index=0
        for i, (true_masks, folder_name) in enumerate(dataloader):
            print(i,end=' ')
            print("avg IoU last image:",jaccard_index)
            optimizer.zero_grad()
            true_masks=true_masks.to(torch.float32).permute(0,4,1,2,3).to(device)
            #true_masks is of dimension here:([16, 49, 22, 160, 240])
            first_11_true_frames = true_masks[:,:,:11,:,:]
            last_11_true_frames=true_masks[:,:,11:,:,:]
            pred_mask=predictor(first_11_true_frames)
            for i in range(0,11):
                #pred mask is of dimension torch.Size([16, 49, 160, 240])
                frame_of_true_mask=true_masks.argmax(dim=1).squeeze()[:, 11+i, :,:]
                loss = criterion(pred_mask,frame_of_true_mask)
                new_correct=torch.sum(pred_mask.argmax(dim=1).squeeze()==frame_of_true_mask)
                new_total=batch_size*160*240
                correct+=new_correct
                total+=new_total
                if i==10:
                    total_last_frame_prediction+=new_total
                    correct_last_frame_prediction+=new_correct
                    jaccard_index=jaccard(pred_mask.argmax(dim=1).squeeze(),frame_of_true_mask)
                total_loss += loss.item()
                if i!=10:
                    next_mask = last_11_true_frames[:, :, i, :, :]
                    next_mask=next_mask.unsqueeze(2)
                    pred_mask= predictor(next_mask)
            
        total_epoch=e+starting_epoch
        print(f"\n\nEpoch: {total_epoch}, Loss: {total_loss}\n, overall prediction accuracy: {correct/total}")
        print(f"\n\nEpoch: {total_epoch}, Prediction accuracy of last frame: {correct_last_frame_prediction/total_last_frame_prediction}")
    print(f"Training took time: {datetime.now() - t1}")
                

In [ ]:
import matplotlib.pyplot as plt

def visualize(pred, actual):

    pred = pred.argmax(dim=1)

    B = pred.shape[0]

    fig, ax = plt.subplots(B // 2, 4, figsize=(160, 240))

    for i in range(B // 2):
        ax[i][0].title.set_text("Actual")
        ax[i][0].imshow(actual[2*i].cpu())
        
        ax[i][1].title.set_text("Predicted")
        ax[i][1].imshow(pred[2*i].cpu())

        ax[i][2].title.set_text("Actual")
        ax[i][2].imshow(actual[2*i+1].cpu())
        
        ax[i][3].title.set_text("Predicted")
        ax[i][3].imshow(pred[2*i+1].cpu())

In [17]:
validate_for_11_frames_input()

0 avg IoU last image: 0


/state/partition1/job-32887502/ipykernel_2858881/530924047.py:138: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.Softmax()(output)


1 avg IoU last image: tensor(0.4574, device='cuda:0')
2 avg IoU last image: tensor(0.4653, device='cuda:0')
3 avg IoU last image: tensor(0.4577, device='cuda:0')
4 avg IoU last image: tensor(0.4670, device='cuda:0')
5 avg IoU last image: tensor(0.4904, device='cuda:0')
6 avg IoU last image: tensor(0.4735, device='cuda:0')
7 avg IoU last image: tensor(0.5057, device='cuda:0')
8 avg IoU last image: tensor(0.4519, device='cuda:0')
9 avg IoU last image: tensor(0.4836, device='cuda:0')
10 avg IoU last image: tensor(0.4626, device='cuda:0')
11 avg IoU last image: tensor(0.4873, device='cuda:0')
12 avg IoU last image: tensor(0.4642, device='cuda:0')
13 avg IoU last image: tensor(0.4897, device='cuda:0')
14 avg IoU last image: tensor(0.4799, device='cuda:0')
15 avg IoU last image: tensor(0.4602, device='cuda:0')
16 avg IoU last image: tensor(0.4715, device='cuda:0')
17 avg IoU last image: tensor(0.4650, device='cuda:0')
18 avg IoU last image: tensor(0.4733, device='cuda:0')
19 avg IoU last ima

In [ ]:
from torch.utils.data import Dataset

class VideoDataset(Dataset):
    def __init__(self, data_path, transform):
        
        self.data_path = data_path
        
        self.files = os.listdir(data_path)
        self.transform = transform
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        
        video_path = f"{self.data_path}/{self.files[i]}"
        frames = []

        for j in range(0, 22):
            frame = Image.open(f"{video_path}/image_{j}.png")
            frames.append(self.transform(frame.convert('RGB')))

        masks = torch.from_numpy(np.load(f"{video_path}/mask.npy"))

        return torch.stack(frames), masks

In [ ]:
valset = VideoDataset("./val", transforms.ToTensor())
val_dataloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

In [ ]:
with torch.no_grad():

    for i, (images, masks) in enumerate(val_dataloader):

        B, N, C, H, W = images.shape

        images = images.permute(0, 2, 1, 3, 4).reshape(-1, C, H, W).cuda()
        masks = masks.reshape(-1, H, W).long().cuda()

        pred_masks = predictor(images)

        j_ind = jaccard(pred_masks, masks)

        print(f"Val Jaccard: {j_ind}")
        
        # print(pred_masks.shape, masks.shape)
        
        visualize(pred_masks[:22], masks[:22])
        break


In [ ]:
!nvidia-smi

In [ ]:
!kill -9 4043746

In [ ]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
train_for_11_frames_input()


In [ ]:
test_dataset = MeanNpyDataset('/scratch/jz3395/val')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8)

ckpt = torch.load('1vidconvlstm_predictor80.pth')
predictor.load_state_dict(ckpt)
predictor.eval()
sum_iou=0


for i, (mean_tensor, folder_name) in enumerate(test_dataloader):
    mean_tensor = mean_tensor.to(torch.float32).permute(0,4,1,2,3)
#     mean_tensor=mean_tensor.unsqueeze(1)
    prev_mean = mean_tensor[:,:,:11,:,:]
    curr_mean=mean_tensor[:,:,11:,:,:]
    pred_mask=predictor(prev_mean)
    print(prev_mean.shape)
    print("pred mas",pred_mask.shape)
    for i in range(0,10):
        next_mask = curr_mean[:, :, i, :, :].to(device)
        next_mask=next_mask.unsqueeze(2)
        pred_mask= predictor(next_mask)
    print(pred_mask.shape)
    for j in range(pred_mask.shape[0]):   
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

        # plot the data on the first subplot
        print(pred_mask[j].shape)
        seg_pred=pred_mask[j].permute(1,2,0).detach().cpu()
        plot_pred=np.argmax(seg_pred, axis=-1)
        
        ax1.imshow(plot_pred.detach().cpu().numpy().squeeze())
        ax1.set_title('predicted')

        # plot the data on the second subplot
        seg_gt=curr_mean[j, :, -1, :, :].permute(1,2,0).detach().cpu()
        plot_gt=np.argmax(seg_gt, axis=-1)
        ax2.imshow(plot_gt.cpu().numpy().squeeze())
        ax2.set_title("GT")

        # adjust the layout of the subplots
        fig.tight_layout()

        
        plt.show()

    break

        
    
    
    
    


In [ ]:
mean_tensor.shape

In [ ]:
mean_tensor=mean_tensor.permute(0,2,3,4,1)

In [ ]:
segmentation_mask = np.argmax(mean_tensor, axis=-1)

In [ ]:
segmentation_mask.shape

In [ ]:
plt.imshow(segmentation_mask[:,21,:,:].squeeze())

In [ ]:
pred_mask.permute(0,2,3,1).shape

In [ ]:
p_pred=pred_mask.permute(0,2,3,1).detach().cpu()
seg_pred=np.argmax(p_pred, axis=-1)

In [ ]:
seg_pred.shape

In [ ]:
plt.imshow(seg_pred.squeeze())

In [ ]:
jaccard = torchmetrics.JaccardIndex(task="multiclass", num_classes=49)

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the output segmentation map obtained from the semantic segmentation model
# output = np.load('output.npy')

# Convert the output segmentation map to a tensor and normalize it
print(output.shape)
output = torch.from_numpy(output.detach().cpu().numpy())
print(output.shape)
output = F.interpolate(output, size=(160,240), mode='bicubic', align_corners=False)
output = F.softmax(output, dim=1)

# Find the class with the highest probability for each pixel
segmentation_map = torch.argmax(output, dim=1).squeeze().cpu().numpy()

# Find the contours of each class in the segmentation map
contours = []
num_classes=49
for i in range(num_classes):
    binary_mask = (segmentation_map == i).astype(np.uint8)
    cnts, hierarchy = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours.append(cnts)

# Draw bounding boxes around the objects of interest for each class
boxes = []
img = cv2.imread('/scratch/pt2295/Dataset_Student/val/video_1031/image_21.png')
for i, cnts in enumerate(contours):
    for cnt in cnts:
        x, y, w, h = cv2.boundingRect(cnt)
        boxes.append([x, y, x + w, y + h, i])
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Visualize the result
plt.imshow(img)
plt.show()

In [ ]:
# temp = torch.randn(1, 6, 21, 601, 601)
# tout = predictor(temp)
def threshold(tensor,thresh):
    return torch.where(tensor < thresh, torch.tensor(0.), torch.tensor(1.));

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
def for_each_img(attn,w_featmap,h_featmap):
#     print(attn.shape)
    attn=attn.unsqueeze(0)
   
    nh = attn.shape[1] # number of head

    # we keep only the output patch attention
    attn = attn[0, :, 0, 1:].reshape(nh, -1)


    # we keep only a certain percentage of the mass
    val, idx = torch.sort(attn)
    val /= torch.sum(val, dim=1, keepdim=True)
    cumval = torch.cumsum(val, dim=1)
    th_attn = cumval > (1 - 0.6)
    idx2 = torch.argsort(idx)
    for head in range(nh):
        th_attn[head] = th_attn[head][idx2[head]]
#     print(w_featmap, h_featmap,th_attn.shape)
    th_attn = th_attn.reshape(nh, w_featmap, h_featmap).float()
    # interpolate
    th_attn = nn.functional.interpolate(th_attn.unsqueeze(0), scale_factor=8, mode="nearest")[0].cpu().numpy()

    attn = attn.reshape(nh, w_featmap, h_featmap)
    attn = nn.functional.interpolate(attn.unsqueeze(0), scale_factor=8, mode="nearest")[0].cpu().numpy()

    # save attn heatmaps
    os.makedirs('output/', exist_ok=True)

    # Saving only last attention layer
    fname = os.path.join('output/',  "temp.png")

    plt.imsave(
        fname=fname,
        arr=sum(attn[i] * 1/attn.shape[0] for i in range(attn.shape[0])),
        cmap="inferno",
        format="jpg"
    )
    img = Image.open('/scratch/pt2295/predictor/output/temp.png')

    # convert the image to grayscale using torchvision.transforms
    gray_transform = transforms.Grayscale()
    gray_img = gray_transform(img)
    t=transforms.ToTensor()(gray_img)
    p=threshold(t,att_thresh)

    
    return p

  

In [ ]:
def transform_att(tensor,w,h):
#     print(tensor.shape)
   
    temp=[]
    for i in range(tensor.shape[0]):
        temp.append(for_each_img(tensor[i],w,h))
    return torch.cat(temp,dim=0)
    
    

In [ ]:
att_thresh=0.4

In [ ]:
from datetime import datetime
def threshold(tensor,thresh):
    return torch.where(tensor < thresh, torch.tensor(0.), torch.tensor(1.));
def train_for_11_frames_input():
    num_epochs = 50

    encoder.eval()
    predictor.train()

    t1 = datetime.now()
    ctr=0
    for e in range(num_epochs):

        total_loss = 0

        for i, (batch, _, _) in enumerate(dataloader):

            optimizer.zero_grad()
            

            print(i, end=", ")
            w, h = batch.shape[1] - batch.shape[1] % 8, batch.shape[2] - batch.shape[2] % 8
            # img = img[:, :w, :h].unsqueeze(0)

            w_featmap = batch.shape[-2] // 8
            h_featmap = batch.shape[-1] // 8   
            with torch.no_grad():
                attentions = encoder.module.student_backbone.module.get_last_selfattention(batch.cuda())
            attentions=transform_att(attentions,w_featmap,h_featmap)
            print(attentions.shape)
            attentions=attentions.unsqueeze(1)

            
            
          
            
            N, C, H, W = attentions.shape
        
            
            B = int(N // num_frames)
            
            attentions = attentions.view(B, num_frames, C, H, W)
           

            prev_attns = attentions[:, :11].permute(0,2,1,3,4)
         
            curr_attns = attentions[:, 11:]
   
            pred_attn= predictor(prev_attns)


            for i in range(0, 10):

                loss = criterion(pred_attn, curr_attns[:,i].to(device))
                

                loss.backward(retain_graph=True)
                optimizer.step()

                total_loss += loss.item()

                next_attn = curr_attns[:,i].unsqueeze(1).permute(0,2,1,3,4)

                pred_attn= predictor(next_attn.to(device))


            loss = criterion(pred_attn, curr_attns[:,-1].to(device))
            print(pred_attn.shape,"predic shape",curr_attns[:,-1].shape)
            visualise(threshold(pred_attn[0,:,:,:],att_thresh),is_pred=True)
            visualise(curr_attns[:,-1][0,:,:,:].to(device))

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        ctr+=1

        torch.save(predictor.state_dict(), 'convlstm_predictor'+str(ctr)+'.pth')

        print(f"\n\nEpoch: {e}, Loss: {total_loss}\n")

    print(f"Training took time: {datetime.now() - t1}")




In [ ]:
# # train_for_21_frames_input()
# 16, 1, 10, 64, 64

In [ ]:
import numpy as np
train_for_11_frames_input()

In [ ]:
predictor.module.first_layer.module.convLSTMcell.module

In [ ]:
import os
import torch.nn as nn
import matplotlib.pyplot as plt

def my_save_image(attn, pred, ctr,is_attn=False):

    print(attn.shape)
    nh = attn.shape[1] # number of head

    # we keep only the output patch attention
    attn = attn[0, :, 0, 1:].reshape(nh, -1)


    # we keep only a certain percentage of the mass
    val, idx = torch.sort(attn)
    val /= torch.sum(val, dim=1, keepdim=True)
    cumval = torch.cumsum(val, dim=1)
    th_attn = cumval > (1 - 0.6)
    idx2 = torch.argsort(idx)
    for head in range(nh):
        th_attn[head] = th_attn[head][idx2[head]]
    print(w_featmap, h_featmap,th_attn.shape)
    th_attn = th_attn.reshape(nh, w_featmap, h_featmap).float()
    # interpolate
    th_attn = nn.functional.interpolate(th_attn.unsqueeze(0), scale_factor=8, mode="nearest")[0].cpu().numpy()

    attn = attn.reshape(nh, w_featmap, h_featmap)
    attn = nn.functional.interpolate(attn.unsqueeze(0), scale_factor=8, mode="nearest")[0].cpu().numpy()

    # save attn heatmaps
    os.makedirs('output/', exist_ok=True)

    # Saving only last attention layer
    fname = os.path.join('output/', "attn-" + str(ctr) + pred + ".png")

    plt.imsave(
        fname=fname,
        arr=sum(attn[i] * 1/attn.shape[0] for i in range(attn.shape[0])),
        cmap="inferno",
        format="jpg"
    )
    print(f"{fname} saved.")

In [ ]:
def visualise(p,is_pred=False):
    np_array = p.detach().cpu().numpy().reshape(160, 240)

    # convert numpy array to PIL Image and convert to grayscale
    pil_image = Image.fromarray(np.uint8(np_array * 255))
    gray_image = pil_image.convert('L')

    # save grayscale PIL Image to disk
    if(is_pred):
        gray_image.save('/scratch/pt2295/predictor/output/gray_image_pred.png')
    else:
        gray_image.save('/scratch/pt2295/predictor/output/gray_image_att.png')

In [ ]:
# from torchvision.utils import save_image
# torch.set_printoptions(profile="full")
ckpt = torch.load('convlstm_predictor49.pth')
predictor.load_state_dict(ckpt)

test_dataset = LightlyDataset(input_dir="/scratch/pt2295/proj/test", transform=transform)
test_dataset.transform = torchvision.transforms.ToTensor()


test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=num_frames,   # batch size 1 for inference
    shuffle=False,  # don't shuffle data during inference
)

encoder.eval()
predictor.eval()

correct = 0
total = 0

ctr = 0
for img, label, fnames in test_dataloader:

    with torch.no_grad():
        print(fnames)
        img = img.cuda()
        print(img.shape)
        w, h = img.shape[1] - img.shape[1] % 8, img.shape[2] - img.shape[2] % 8
        # img = img[:, :w, :h].unsqueeze(0)

        w_featmap = img.shape[-2] // 8
        h_featmap = img.shape[-1] // 8

        attentions = encoder.module.student_backbone.module.get_last_selfattention(img.cuda())
        temp=attentions
        attentions=transform_att(attentions,w_featmap,h_featmap)
        attentions=attentions.unsqueeze(1)


        N, C, H, W = attentions.shape

        prev_attns = attentions[:11].view(1, 11, C, H, W).permute(0, 2, 1, 3, 4)


        pred_attn = predictor(prev_attns)

        for i in range(11, 21):

            next_attn = attentions[i].view(1, 1, C, H, W).permute(0,2,1,3,4)

            pred_attn= predictor(next_attn)
            

            

        correct += torch.sum(pred_attn.view(1, -1) == attentions[-1].view(1, -1).to(device))

        total += 6*601*601

        my_save_image(attentions[-1], "_3321_o", ctr)
        my_save_image(pred_attn, "_3321_p", ctr)
        print("##############")
#         print(attentions[-1])
        print("*************")
#         print(pred_attn)
        
        print("how many equal",(threshold(pred_attn,0.2).cuda() == attentions[-1].squeeze(0).cuda()).sum().item())
        print("1 in attention",(attentions[-1] == 1).sum().item())
        print("how many in pred 1",(threshold(pred_attn,0.2) == 1).sum().item())
        print("mae")

        ctr += 1
        

print(f"Accuracy: {correct/total}")

In [ ]:
# import torch
# import torchvision.utils as vutils
# vutils.save_image(threshold(pred_attn), 'my_predsttn.png', normalize=True, range=(0, 255))
# vutils.save_image(attentions[-1].squeeze(0), 'my_attn.png', normalize=True, range=(0, 255))


In [ ]:
vutils.save_image(temp[-1], 'my_tttttempattn.png', normalize=True, range=(0, 1), nrow=1, pad_value=1)

In [ ]:
temp[-1].shape

In [ ]:
temp[-1].unsqueeze(0).shape

In [ ]:
my_save_image(temp[-1].unsqueeze(0),"blah","2")

In [ ]:
# torch.abs(threshold(pred_attn) - attentions[-1].squeeze(0))

In [ ]:
# attentions[-1]

In [ ]:
pred_attn.squeeze(0).shape

In [ ]:
pred_attn.shape

In [ ]:
threshold(pred_attn[:,:,0,:])

In [ ]:
# torch.sigmoid(pred_attn[0,0,0])

In [ ]:
loss = criterion(pred_attn, torch.sigmoid(attentions[-1].unsqueeze(0)).to(device))
loss.item()

In [ ]:
pred_attn.shape, attentions[-1].shape

In [ ]:
datetime.now() - t1

In [ ]:
9714 / 60 / 60

In [ ]:
total_loss

In [ ]:
import torch
import torchvision.utils as vutils

# create a grayscale tensor
x = torch.rand(1, 601, 601)

# save tensor as a grayscale image
vutils.save_image(x, 'my_tensor_image.png', normalize=True, range=(0, 1), nrow=1, pad_value=1)


In [ ]:
x

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image


# define the image file path
img_file = '/scratch/pt2295/predictor/output/attn-2blah.png'

# create a transformation to convert the image to a tensor
transform = transforms.ToTensor()

# load the image as a tensor
img_tensor = transform(Image.open(img_file))

# print the tensor shape
print(img_tensor.shape)


In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# load the image as a PIL Image object
img = Image.open('/scratch/pt2295/predictor/output/attn-2blah.png')

# convert the image to grayscale using torchvision.transforms
gray_transform = transforms.Grayscale()
gray_img = gray_transform(img)

# save the grayscale image as a file
gray_img.save('/scratch/pt2295/predictor/output/att2blah_gray.jpg')


In [ ]:
t=transforms.ToTensor()(gray_img)
torch.sum(t > 0.2)

In [ ]:
t.shape

In [ ]:
p=threshold(t,0.2)
import numpy as np

In [ ]:
np_array = p.numpy().reshape(160, 240)

# convert numpy array to PIL Image and convert to grayscale
pil_image = Image.fromarray(np.uint8(np_array * 255))
gray_image = pil_image.convert('L')

# save grayscale PIL Image to disk
gray_image.save('/scratch/pt2295/predictor/output/grayscale_image.png')

In [ ]:
p.shape

In [ ]:
plt.hist(t.flatten().numpy(), bins=50)
plt.show()

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# load the image as a PIL Image object
img = Image.open('/scratch/pt2295/predictor/output/attn-2blah.png')

# convert the image to grayscale using torchvision.transforms
gray_transform = transforms.Grayscale()

gray_img = gray_transform(img)

# save the grayscale image as a file
gray_img.save('/scratch/pt2295/predictor/output/ttatt2blah_gray.jpg')